In [17]:
import numpy as np
def function(x1, x2, x3):
    return x1**2 + x2**2 + x3**2

def constraints(x1, x2, x3):
    h1 = (x1**2)/4 + (x2**2)/5 + (x3**2)/25 -1
    h2 = x1+x2-x3
    return h1, h2

def reduced_gradient(x1, x2, x3):
    df_ds = np.vstack((2 * x2, 2 * x3))
    par_df_dd = np.array([2 * x1])
    dh_ds = np.vstack((np.hstack(((2/5) * x2, (2/25) * x3)), np.array([[1, -1]], dtype=float)))
    dh_dd = np.vstack(((1 / 2) * x1, 1.))
    dh_ds_inv = np.linalg.inv(dh_ds)
    df_dd = par_df_dd - np.matmul(df_ds.T, np.matmul(dh_ds_inv, dh_dd))
    return df_ds, par_df_dd, dh_ds, dh_dd, dh_ds_inv, df_dd

def line_search(x1, x2, x3):
    alpha = 1.0
    b = 0.5
    t = 0.3

    def phi(x1, x2, x3, alpha):
        df_ds, par_df_dd, dh_ds, dh_dd, dh_ds_inv,df_dd = reduced_gradient(x1, x2, x3)
        return function(x1, x2, x3) - alpha * t * np.matmul(df_dd, df_dd.T)  

    def f(x1, x2, x3, alpha):
        x1 = x1 - alpha * df_dd.flatten()
        ds = np.matmul(np.matmul(dh_ds_inv, dh_dd), df_dd.T).flatten()
        x2 = x2 + alpha * ds[0]
        x3 = x3 + alpha * ds[1]
        return function(x1, x2, x3)

    phiv = phi(x1, x2, x3, alpha)
    fv = f(x1, x2, x3, alpha)
    
    while phiv < fv:
        alpha = 0.5 * alpha
        phiv = phi(x1, x2, x3, alpha)
        fv = f(x1, x2, x3, alpha)
    return alpha

def solve_function(x1, x2, x3):
    error = 1e-3
    h1, h2 = constraints(x1, x2, x3)
    h = np.vstack((h1, h2))
    h_norm = np.linalg.norm(h)

    while h_norm >= error:
        df_ds, par_df_dd, dh_ds, dh_dd, dh_ds_inv, df_dd = reduced_gradient(x1, x2, x3)
        ds = np.matmul(dh_ds_inv, h)
        x2 = x2 - ds[0]
        x3 = x3 - ds[1]
        h1, h2 = constraints(x1, x2, x3)
        h = np.vstack((h1, h2))
        h_norm = np.linalg.norm(h)
    return x1, x2, x3, h_norm

eps = 1e-3 
x1 = np.array([1.0])
x2 = np.array([1.0])
x3 = np.array([1.0])

x1, x2, x3, _ = solve_function(x1, x2, x3)
df_ds, par_df_dd, dh_ds, dh_dd, dh_ds_inv, df_dd = reduced_gradient(x1, x2, x3)
df_dd_norm = np.linalg.norm(df_dd)

while df_dd_norm >= eps:
    alpha = line_search(x1, x2, x3)
    df_ds, par_df_dd, dh_ds, dh_dd, dh_ds_inv, df_dd = reduced_gradient(x1, x2, x3)
    x1 = x1 - alpha * df_dd.flatten()
    df_ds, par_df_dd, dh_ds, dh_dd, dh_ds_inv, df_dd = reduced_gradient(x1, x2, x3)
    ds = alpha * np.matmul(np.matmul(dh_ds_inv, dh_dd), df_dd.T).flatten()
    x2 = x2 + ds[0]
    x3 = x3 + ds[1]
    x1, x2, x3, _ = solve_function(x1, x2, x3)
    df_ds, par_df_dd, dh_ds, dh_dd, dh_ds_inv, df_dd = reduced_gradient(x1, x2, x3)
    df_dd_norm = np.linalg.norm(df_dd)

print('x1={}\nx2={}\nx3={}'.format(x1, x2, x3))

x1=[-1.57327871]
x2=[1.3772831]
x3=[-0.19658949]
